In [20]:
! pip install spacy
! python -m spacy download en_core_web_sm
import spacy


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 44.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
! pip install pandarallel
from pandarallel import pandarallel

  Using cached pandarallel-1.6.4-py3-none-any.whl

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import time
import pandas as pd
import re
import requests

In [5]:
import numpy as np
import scipy
import ujson as json
import boto3
import s3fs
from multiprocessing import Process, Manager
from multiprocessing.pool import ThreadPool, Pool


In [6]:
# loading in high peace 1m dataset 

#hp_df = pd.read_csv('s3://compressed-data-sample/compressed1m_hp.csv', 
  #                        dtype={"wordCount": 'int16', "country":'category'})

In [7]:
# importing 

In [8]:
hp_clean = pd.read_csv('s3://compressed-data-sample/processed_hp.csv')

In [23]:
lp_clean = pd.read_csv('s3://compressed-data-sample/processed_lp.csv')

In [24]:
# loading in low peace 1m dataset 

#lp_df = pd.read_csv('s3://compressed-data-sample/compressed1m_lp.csv',
  #                   dtype={"wordCount": 'int16', "country":'category'})

In [25]:
hp_clean.shape

(983589, 7)

In [26]:
lp_clean.shape

(996134, 6)

In [27]:
hp_clean['peace/non-peace'] = 1

In [28]:
lp_clean['peace/non-peace'] = 0

In [29]:
hp_clean.head(1)

,Unnamed: 0,title,content,wordCount,country,content_cleaned,peace/non-peace
0,0,MONTHLY SECTOR REPORT: S&P/ASX EMERGING COMPAN...,AUSTRALIAN MONTHLY INDEX REPORT\n\nThe S&P/ASX...,2405,Australia,MONTHLY SECTOR REPORT : / ASX EMERGING COMPANI...,1


In [30]:
hp_clean.shape

(983589, 7)

In [31]:
lp_clean.shape

(996134, 7)

In [32]:
hp_lp_concat = pd.concat([hp_clean, lp_clean], ignore_index=True)

In [33]:
hp_lp_concat.head()

,Unnamed: 0,title,content,wordCount,country,content_cleaned,peace/non-peace
0,0,MONTHLY SECTOR REPORT: S&P/ASX EMERGING COMPAN...,AUSTRALIAN MONTHLY INDEX REPORT\n\nThe S&P/ASX...,2405,Australia,MONTHLY SECTOR REPORT : / ASX EMERGING COMPANI...,1
1,1,PAPRIKA CLUB RESTAURANT,THE Paprika Club Restaurant offers freshly pre...,120,Australia,"RESTAURANT \n offers freshly prepared, delicio...",1
2,2,Bone implants to be made from 3D printed salt,ABSTRACT\n\n\n\nResearchers have developed a n...,545,Australia,Bone implants to be made from 3D printed salt ...,1
3,3,"Yes, you're entitled to a free portrait of the...",It's a truth universally acknowledged that a h...,598,Australia,"Yes, you're entitled to free portrait of the. ...",1
4,4,Why is Hemsworth wearing nail polish?,If you thought nail polish was only for women ...,729,Australia,Why is wearing nail polish? \n If you thought ...,1


In [34]:
hp_lp_concat['peace/non-peace'].value_counts()

0    996134
1    983589
Name: peace/non-peace, dtype: int64

In [35]:
hp_lp_concat.reset_index(drop=True,inplace = True)

In [36]:
hp_lp_concat = hp_lp_concat.drop(['Unnamed: 0'],axis=1)

In [37]:
hp_lp_concat

,title,content,wordCount,country,content_cleaned,peace/non-peace
0,MONTHLY SECTOR REPORT: S&P/ASX EMERGING COMPAN...,AUSTRALIAN MONTHLY INDEX REPORT\n\nThe S&P/ASX...,2405,Australia,MONTHLY SECTOR REPORT : / ASX EMERGING COMPANI...,1
1,PAPRIKA CLUB RESTAURANT,THE Paprika Club Restaurant offers freshly pre...,120,Australia,"RESTAURANT \n offers freshly prepared, delicio...",1
2,Bone implants to be made from 3D printed salt,ABSTRACT\n\n\n\nResearchers have developed a n...,545,Australia,Bone implants to be made from 3D printed salt ...,1
3,"Yes, you're entitled to a free portrait of the...",It's a truth universally acknowledged that a h...,598,Australia,"Yes, you're entitled to free portrait of the. ...",1
4,Why is Hemsworth wearing nail polish?,If you thought nail polish was only for women ...,729,Australia,Why is wearing nail polish? \n If you thought ...,1
...,...,...,...,...,...,...
1979718,"Intel Awarded Patent for Systems, Methods and ...",FULL TEXT\n\n\n\nPublication Name: Software Pa...,241,India,\n FULL TEXT \n\n\n\n Publication Name : Softw...,0
1979719,Morning Alert: Williamson Magor & Co keeps ris...,INDIAN INTRA-DAY STOCK REPORT\n\nWilliamson Ma...,680,India,Morning Alert : keeps rising : up 22.0% in 6 d...,0
1979720,"LJUBLJANSKE MLEKARNE, mlekarska industrija, d....","Slovenia, Nov. 14 -- Slovenia based LJUBLJANSK...",68,India,"LJUBLJANSKE MLEKARNE, industrija, d.o.o. contr...",0
1979721,STEEL AUTHORITY OF INDIA LIMITED (SAIL) Provid...,"India ,April 10 -- Tender No. CCNW/PR11300076...",51,India,STEEL AUTHORITY OF INDIA LIMITED (SAIL) Provid...,0


In [41]:
us2gb = {
  "Africanisation": "Africanization",
  "Africanise": "Africanize",
  "Americanisation": "Americanization",
  "Americanise": "Americanize",
  "Arabise": "Arabize",
  "Finlandisation": "Finlandization",
  "africanisation": "africanization",
  "africanise": "africanize",
  "americanisation": "americanization",
  "americanise": "americanize",
  "arabise": "arabize",
  "finlandisation": "finlandization",
  "accessorise": "accessorize",
  "accessorised": "accessorized",
  "accessorises": "accessorizes",
  "accessorising": "accessorizing",
  "acclimatisation": "acclimatization",
  "acclimatise": "acclimatize",
  "acclimatised": "acclimatized",
  "acclimatises": "acclimatizes",
  "acclimatising": "acclimatizing",
  "accoutrements": "accouterments",
  "aeon": "eon",
  "aeons": "eons",
  "aerogramme": "aerogram",
  "aerogrammes": "aerograms",
  "aeroplane": "airplane",
  "aeroplanes": "airplanes",
  "aesthete": "esthete",
  "aesthetes": "esthetes",
  "aesthetic": "esthetic",
  "aesthetically": "esthetically",
  "aesthetics": "esthetics",
  "aetiology": "etiology",
  "ageing": "aging",
  "aggrandise": "aggrandize",
  "aggrandisement": "aggrandizement",
  "agonise": "agonize",
  "agonised": "agonized",
  "agonises": "agonizes",
  "agonising": "agonizing",
  "agonisingly": "agonizingly",
  "allegorise": "allegorize",
  "almanack": "almanac",
  "almanacks": "almanacs",
  "aluminium": "aluminum",
  "amortisable": "amortizable",
  "amortisation": "amortization",
  "amortisations": "amortizations",
  "amortise": "amortize",
  "amortised": "amortized",
  "amortises": "amortizes",
  "amortising": "amortizing",
  "amoxycillin": "amoxicillin",
  "amphitheatre": "amphitheater",
  "amphitheatres": "amphitheaters",
  "anaemia": "anemia",
  "anaemic": "anemic",
  "anaesthesia": "anesthesia",
  "anaesthetic": "anesthetic",
  "anaesthetics": "anesthetics",
  "anaesthetise": "anesthetize",
  "anaesthetised": "anesthetized",
  "anaesthetises": "anesthetizes",
  "anaesthetising": "anesthetizing",
  "anaesthetist": "anesthetist",
  "anaesthetists": "anesthetists",
  "anaesthetize": "anesthetize",
  "anaesthetized": "anesthetized",
  "anaesthetizes": "anesthetizes",
  "anaesthetizing": "anesthetizing",
  "analogue": "analog",
  "analogues": "analogs",
  "analyse": "analyze",
  "analysed": "analyzed",
  "analyses": "analyzes",
  "analysing": "analyzing",
  "ancle": "ankle",
  "anglicise": "anglicize",
  "anglicised": "anglicized",
  "anglicises": "anglicizes",
  "anglicising": "anglicizing",
  "annexe": "annex",
  "annualised": "annualized",
  "antagonise": "antagonize",
  "antagonised": "antagonized",
  "antagonises": "antagonizes",
  "antagonising": "antagonizing",
  "apologise": "apologize",
  "apologised": "apologized",
  "apologises": "apologizes",
  "apologising": "apologizing",
  "apophthegm": "apothegm",
  "apostatise": "apostatize",
  "appal": "appall",
  "appals": "appalls",
  "appetiser": "appetizer",
  "appetisers": "appetizers",
  "appetising": "appetizing",
  "appetisingly": "appetizingly",
  "arbour": "arbor",
  "arbours": "arbors",
  "archaeological": "archeological",
  "archaeologically": "archeologically",
  "archaeologist": "archeologist",
  "archaeologists": "archeologists",
  "archaeology": "archeology",
  "ardour": "ardor",
  "armour": "armor",
  "armoured": "armored",
  "armourer": "armorer",
  "armourers": "armorers",
  "armouries": "armories",
  "armoury": "armory",
  "artefact": "artifact",
  "artefacts": "artifacts",
  "atomise": "atomize",
  "atomiser": "atomizer",
  "attitudinise": "attitudinize",
  "authorisation": "authorization",
  "authorise": "authorize",
  "authorised": "authorized",
  "authorises": "authorizes",
  "authorising": "authorizing",
  "autolyse": "autolyze",
  "axe": "ax",
  "axiomatisation": "axiomatization",
  "backpedalled": "backpedaled",
  "backpedalling": "backpedaling",
  "backtenter": "backtender",
  "baist": "baste",
  "bannister": "banister",
  "bannisters": "banisters",
  "baptise": "baptize",
  "baptised": "baptized",
  "baptises": "baptizes",
  "baptising": "baptizing",
  "bastardise": "bastardize",
  "bastardised": "bastardized",
  "bastardises": "bastardizes",
  "bastardising": "bastardizing",
  "battleaxe": "battleax",
  "baulk": "balk",
  "baulked": "balked",
  "baulking": "balking",
  "baulks": "balks",
  "bedevilled": "bedeviled",
  "bedevilling": "bedeviling",
  "behaviour": "behavior",
  "behavioural": "behavioral",
  "behaviourism": "behaviorism",
  "behaviourist": "behaviorist",
  "behaviourists": "behaviorists",
  "behaviours": "behaviors",
  "behove": "behoove",
  "behoved": "behooved",
  "behoves": "behooves",
  "bejewelled": "bejeweled",
  "belabour": "belabor",
  "belaboured": "belabored",
  "belabouring": "belaboring",
  "belabours": "belabors",
  "bevelled": "beveled",
  "bevvies": "bevies",
  "bevvy": "bevy",
  "biassed": "biased",
  "biassing": "biasing",
  "bingeing": "binging",
  "botanise": "botanize",
  "bougainvillaea": "bougainvillea",
  "bougainvillaeas": "bougainvilleas",
  "bowdlerise": "bowdlerize",
  "bowdlerised": "bowdlerized",
  "bowdlerises": "bowdlerizes",
  "bowdlerising": "bowdlerizing",
  "breathalyse": "breathalyze",
  "breathalysed": "breathalyzed",
  "breathalyser": "breathalyzer",
  "breathalysers": "breathalyzers",
  "breathalyses": "breathalyzes",
  "breathalysing": "breathalyzing",
  "brutalise": "brutalize",
  "brutalised": "brutalized",
  "brutalises": "brutalizes",
  "brutalising": "brutalizing",
  "buses": "busses",
  "busing": "bussing",
  "caesarean": "cesarean",
  "caesareans": "cesareans",
  "caesium": "cesium",
  "calibre": "caliber",
  "calibres": "calibers",
  "calliper": "caliper",
  "callipers": "calipers",
  "callisthenics": "calisthenics",
  "canalise": "canalize",
  "canalised": "canalized",
  "canalises": "canalizes",
  "canalising": "canalizing",
  "cancellation": "cancelation",
  "cancellations": "cancelations",
  "cancelled": "canceled",
  "cancelling": "canceling",
  "candour": "candor",
  "cannibalise": "cannibalize",
  "cannibalised": "cannibalized",
  "cannibalises": "cannibalizes",
  "cannibalising": "cannibalizing",
  "canonise": "canonize",
  "canonised": "canonized",
  "canonises": "canonizes",
  "canonising": "canonizing",
  "capitalise": "capitalize",
  "capitalised": "capitalized",
  "capitalises": "capitalizes",
  "capitalising": "capitalizing",
  "caramelise": "caramelize",
  "caramelised": "caramelized",
  "caramelises": "caramelizes",
  "caramelising": "caramelizing",
  "carbonise": "carbonize",
  "carbonised": "carbonized",
  "carbonises": "carbonizes",
  "carbonising": "carbonizing",
  "carcase": "carcass",
  "carolled": "caroled",
  "carolling": "caroling",
  "catalogue": "catalog",
  "catalogued": "cataloged",
  "catalogues": "catalogs",
  "cataloguing": "cataloging",
  "catalyse": "catalyze",
  "catalysed": "catalyzed",
  "catalyses": "catalyzes",
  "catalysing": "catalyzing",
  "categorise": "categorize",
  "categorised": "categorized",
  "categorises": "categorizes",
  "categorising": "categorizing",
  "cauterise": "cauterize",
  "cauterised": "cauterized",
  "cauterises": "cauterizes",
  "cauterising": "cauterizing",
  "cavilled": "caviled",
  "cavilling": "caviling",
  "centigramme": "centigram",
  "centigrammes": "centigrams",
  "centilitre": "centiliter",
  "centilitres": "centiliters",
  "centimetre": "centimeter",
  "centimetres": "centimeters",
  "centralise": "centralize",
  "centralised": "centralized",
  "centralises": "centralizes",
  "centralising": "centralizing",
  "centre": "center",
  "centred": "centered",
  "centrefold": "centerfold",
  "centrefolds": "centerfolds",
  "centrepiece": "centerpiece",
  "centrepieces": "centerpieces",
  "centres": "centers",
  "channelled": "channeled",
  "channelling": "channeling",
  "characterisation": "characterization",
  "characterise": "characterize",
  "characterised": "characterized",
  "characterises": "characterizes",
  "characterising": "characterizing",
  "chequebook": "checkbook",
  "chequebooks": "checkbooks",
  "chequered": "checkered",
  "chilli": "chili",
  "chimaera": "chimera",
  "chimaeras": "chimeras",
  "chimaeric": "chimeric",
  "chimaerism": "chimerism",
  "chiselled": "chiseled",
  "chiselling": "chiseling",
  "circularise": "circularize",
  "circularised": "circularized",
  "circularises": "circularizes",
  "circularising": "circularizing",
  "civilisation": "civilization",
  "civilise": "civilize",
  "civilised": "civilized",
  "civilises": "civilizes",
  "civilising": "civilizing",
  "clamour": "clamor",
  "clamoured": "clamored",
  "clamouring": "clamoring",
  "clamours": "clamors",
  "clangour": "clangor",
  "clarinettist": "clarinetist",
  "clarinettists": "clarinetists",
  "collectivise": "collectivize",
  "collectivised": "collectivized",
  "collectivises": "collectivizes",
  "collectivising": "collectivizing",
  "colonisation": "colonization",
  "colonise": "colonize",
  "colonised": "colonized",
  "coloniser": "colonizer",
  "colonisers": "colonizers",
  "colonises": "colonizes",
  "colonising": "colonizing",
  "colour": "color",
  "colourant": "colorant",
  "colourants": "colorants",
  "coloured": "colored",
  "coloureds": "coloreds",
  "colourful": "colorful",
  "colourfully": "colorfully",
  "colouring": "coloring",
  "colourize": "colorize",
  "colourized": "colorized",
  "colourizes": "colorizes",
  "colourizing": "colorizing",
  "colourless": "colorless",
  "colours": "colors",
  "commercialise": "commercialize",
  "commercialised": "commercialized",
  "commercialises": "commercializes",
  "commercialising": "commercializing",
  "compartmentalise": "compartmentalize",
  "compartmentalised": "compartmentalized",
  "compartmentalises": "compartmentalizes",
  "compartmentalising": "compartmentalizing",
  "computerise": "computerize",
  "computerised": "computerized",
  "computerises": "computerizes",
  "computerising": "computerizing",
  "conceptualise": "conceptualize",
  "conceptualised": "conceptualized",
  "conceptualises": "conceptualizes",
  "conceptualising": "conceptualizing",
  "connexion": "connection",
  "connexional": "connectional",
  "connexions": "connections",
  "contextualise": "contextualize",
  "contextualised": "contextualized",
  "contextualises": "contextualizes",
  "contextualising": "contextualizing",
  "conveyorise": "conveyorize",
  "cosier": "cozier",
  "cosies": "cozies",
  "cosiest": "coziest",
  "cosily": "cozily",
  "cosiness": "coziness",
  "cosy": "cozy",
  "councillor": "councilor",
  "councillors": "councilors",
  "counselled": "counseled",
  "counselling": "counseling",
  "counsellor": "counselor",
  "counsellors": "counselors",
  "crenellated": "crenelated",
  "creolise": "creolize",
  "criminalise": "criminalize",
  "criminalised": "criminalized",
  "criminalises": "criminalizes",
  "criminalising": "criminalizing",
  "criticise": "criticize",
  "criticised": "criticized",
  "criticises": "criticizes",
  "criticising": "criticizing",
  "crueller": "crueler",
  "cruellest": "cruelest",
  "crystallisation": "crystallization",
  "crystallise": "crystallize",
  "crystallised": "crystallized",
  "crystallises": "crystallizes",
  "crystallising": "crystallizing",
  "cudgelled": "cudgeled",
  "cudgelling": "cudgeling",
  "customise": "customize",
  "customised": "customized",
  "customises": "customizes",
  "customising": "customizing",
  "decentralisation": "decentralization",
  "decentralise": "decentralize",
  "decentralised": "decentralized",
  "decentralises": "decentralizes",
  "decentralising": "decentralizing",
  "decriminalisation": "decriminalization",
  "decriminalise": "decriminalize",
  "decriminalised": "decriminalized",
  "decriminalises": "decriminalizes",
  "decriminalising": "decriminalizing",
  "defence": "defense",
  "defenceless": "defenseless",
  "defences": "defenses",
  "deflexion": "deflection",
  "dehumanisation": "dehumanization",
  "dehumanise": "dehumanize",
  "dehumanised": "dehumanized",
  "dehumanises": "dehumanizes",
  "dehumanising": "dehumanizing",
  "demeanour": "demeanor",
  "demilitarisation": "demilitarization",
  "demilitarise": "demilitarize",
  "demilitarised": "demilitarized",
  "demilitarises": "demilitarizes",
  "demilitarising": "demilitarizing",
  "demobilisation": "demobilization",
  "demobilise": "demobilize",
  "demobilised": "demobilized",
  "demobilises": "demobilizes",
  "demobilising": "demobilizing",
  "democratisation": "democratization",
  "democratise": "democratize",
  "democratised": "democratized",
  "democratises": "democratizes",
  "democratising": "democratizing",
  "demonise": "demonize",
  "demonised": "demonized",
  "demonises": "demonizes",
  "demonising": "demonizing",
  "demoralisation": "demoralization",
  "demoralise": "demoralize",
  "demoralised": "demoralized",
  "demoralises": "demoralizes",
  "demoralising": "demoralizing",
  "denationalisation": "denationalization",
  "denationalise": "denationalize",
  "denationalised": "denationalized",
  "denationalises": "denationalizes",
  "denationalising": "denationalizing",
  "deodorise": "deodorize",
  "deodorised": "deodorized",
  "deodorises": "deodorizes",
  "deodorising": "deodorizing",
  "depersonalise": "depersonalize",
  "depersonalised": "depersonalized",
  "depersonalises": "depersonalizes",
  "depersonalising": "depersonalizing",
  "deputise": "deputize",
  "deputised": "deputized",
  "deputises": "deputizes",
  "deputising": "deputizing",
  "desensitisation": "desensitization",
  "desensitise": "desensitize",
  "desensitised": "desensitized",
  "desensitises": "desensitizes",
  "desensitising": "desensitizing",
  "despatch": "dispatch",
  "destabilisation": "destabilization",
  "destabilise": "destabilize",
  "destabilised": "destabilized",
  "destabilises": "destabilizes",
  "destabilising": "destabilizing",
  "dialled": "dialed",
  "dialling": "dialing",
  "dialogue": "dialog",
  "dialogues": "dialogs",
  "diarrhoea": "diarrhea",
  "digitise": "digitize",
  "digitised": "digitized",
  "digitises": "digitizes",
  "digitising": "digitizing",
  "disc": "disk",
  "discolour": "discolor",
  "discoloured": "discolored",
  "discolouring": "discoloring",
  "discolours": "discolors",
  "discs": "disks",
  "disembowelled": "disemboweled",
  "disembowelling": "disemboweling",
  "disfavour": "disfavor",
  "dishevelled": "disheveled",
  "dishonour": "dishonor",
  "dishonourable": "dishonorable",
  "dishonourably": "dishonorably",
  "dishonoured": "dishonored",
  "dishonouring": "dishonoring",
  "dishonours": "dishonors",
  "disorganisation": "disorganization",
  "disorganised": "disorganized",
  "distil": "distill",
  "distils": "distills",
  "doorsman": "doorman",
  "doyley": "doily",
  "dramatisation": "dramatization",
  "dramatisations": "dramatizations",
  "dramatise": "dramatize",
  "dramatised": "dramatized",
  "dramatises": "dramatizes",
  "dramatising": "dramatizing",
  "draught": "draft",
  "draughtboard": "draftboard",
  "draughtboards": "draftboards",
  "draughtier": "draftier",
  "draughtiest": "draftiest",
  "draughtsman": "draftsman",
  "draughtsmanship": "draftsmanship",
  "draughtsmen": "draftsmen",
  "draughtswoman": "draftswoman",
  "draughtswomen": "draftswomen",
  "draughty": "drafty",
  "drivelled": "driveled",
  "drivelling": "driveling",
  "duelled": "dueled",
  "duelling": "dueling",
  "dyspnoea": "dyspnea",
  "economise": "economize",
  "economised": "economized",
  "economises": "economizes",
  "economising": "economizing",
  "editorialise": "editorialize",
  "editorialised": "editorialized",
  "editorialises": "editorializes",
  "editorialising": "editorializing",
  "edoema": "edema",
  "embolisation": "embolization",
  "empathise": "empathize",
  "empathised": "empathized",
  "empathises": "empathizes",
  "empathising": "empathizing",
  "emphasise": "emphasize",
  "emphasised": "emphasized",
  "emphasises": "emphasizes",
  "emphasising": "emphasizing",
  "enamelled": "enameled",
  "enamelling": "enameling",
  "enamour": "enamor",
  "enamoured": "enamored",
  "encyclopaedia": "encyclopedia",
  "encyclopaedias": "encyclopedias",
  "encyclopaedic": "encyclopedic",
  "endeavour": "endeavor",
  "endeavoured": "endeavored",
  "endeavouring": "endeavoring",
  "endeavours": "endeavors",
  "energise": "energize",
  "energised": "energized",
  "energises": "energizes",
  "energising": "energizing",
  "enrol": "enroll",
  "enrols": "enrolls",
  "enthral": "enthrall",
  "enthrals": "enthralls",
  "epaulette": "epaulet",
  "epaulettes": "epaulets",
  "epicentre": "epicenter",
  "epicentres": "epicenters",
  "epilogue": "epilog",
  "epilogues": "epilogs",
  "epitomise": "epitomize",
  "epitomised": "epitomized",
  "epitomises": "epitomizes",
  "epitomising": "epitomizing",
  "equalisation": "equalization",
  "equalise": "equalize",
  "equalised": "equalized",
  "equaliser": "equalizer",
  "equalisers": "equalizers",
  "equalises": "equalizes",
  "equalising": "equalizing",
  "ergotised": "ergotized",
  "eulogise": "eulogize",
  "eulogised": "eulogized",
  "eulogises": "eulogizes",
  "eulogising": "eulogizing",
  "evangelise": "evangelize",
  "evangelised": "evangelized",
  "evangelises": "evangelizes",
  "evangelising": "evangelizing",
  "exorcise": "exorcize",
  "exorcised": "exorcized",
  "exorcises": "exorcizes",
  "exorcising": "exorcizing",
  "extemporisation": "extemporization",
  "extemporise": "extemporize",
  "extemporised": "extemporized",
  "extemporises": "extemporizes",
  "extemporising": "extemporizing",
  "exteriorise": "exteriorize",
  "externalisation": "externalization",
  "externalisations": "externalizations",
  "externalise": "externalize",
  "externalised": "externalized",
  "externalises": "externalizes",
  "externalising": "externalizing",
  "factorise": "factorize",
  "factorised": "factorized",
  "factorises": "factorizes",
  "factorising": "factorizing",
  "faecal": "fecal",
  "faeces": "feces",
  "familiarisation": "familiarization",
  "familiarise": "familiarize",
  "familiarised": "familiarized",
  "familiarises": "familiarizes",
  "familiarising": "familiarizing",
  "fantasise": "fantasize",
  "fantasised": "fantasized",
  "fantasises": "fantasizes",
  "fantasising": "fantasizing",
  "favour": "favor",
  "favourable": "favorable",
  "favourably": "favorably",
  "favoured": "favored",
  "favouring": "favoring",
  "favourite": "favorite",
  "favourites": "favorites",
  "favouritism": "favoritism",
  "favours": "favors",
  "featurisation": "featurization",
  "featurise": "featurize",
  "featurised": "featurized",
  "feminise": "feminize",
  "feminised": "feminized",
  "feminises": "feminizes",
  "feminising": "feminizing",
  "fertilisation": "fertilization",
  "fertilise": "fertilize",
  "fertilised": "fertilized",
  "fertiliser": "fertilizer",
  "fertilisers": "fertilizers",
  "fertilises": "fertilizes",
  "fertilising": "fertilizing",
  "fervour": "fervor",
  "fibre": "fiber",
  "fibreglass": "fiberglass",
  "fibres": "fibers",
  "fictionalisation": "fictionalization",
  "fictionalisations": "fictionalizations",
  "fictionalise": "fictionalize",
  "fictionalised": "fictionalized",
  "fictionalises": "fictionalizes",
  "fictionalising": "fictionalizing",
  "fillet": "filet",
  "filleted": "fileted",
  "filleting": "fileting",
  "fillets": "filets",
  "finalisation": "finalization",
  "finalise": "finalize",
  "finalised": "finalized",
  "finalises": "finalizes",
  "finalising": "finalizing",
  "flautist": "flutist",
  "flautists": "flutists",
  "flavour": "flavor",
  "flavoured": "flavored",
  "flavouring": "flavoring",
  "flavourings": "flavorings",
  "flavourless": "flavorless",
  "flavours": "flavors",
  "flavoursome": "flavorsome",
  "foetal": "fetal",
  "foetid": "fetid",
  "foeto": "feto-",
  "foetus": "fetus",
  "foetuses": "fetuses",
  "formalisation": "formalization",
  "formalise": "formalize",
  "formalised": "formalized",
  "formalises": "formalizes",
  "formalising": "formalizing",
  "fossilisation": "fossilization",
  "fossilise": "fossilize",
  "fossilised": "fossilized",
  "fossilises": "fossilizes",
  "fossilising": "fossilizing",
  "fraternisation": "fraternization",
  "fraternise": "fraternize",
  "fraternised": "fraternized",
  "fraternises": "fraternizes",
  "fraternising": "fraternizing",
  "fulfil": "fulfill",
  "fulfilment": "fulfillment",
  "fulfils": "fulfills",
  "funnelled": "funneled",
  "funnelling": "funneling",
  "galvanise": "galvanize",
  "galvanised": "galvanized",
  "galvanises": "galvanizes",
  "galvanising": "galvanizing",
  "gambolled": "gamboled",
  "gambolling": "gamboling",
  "gaolbird": "jailbird",
  "gaolbirds": "jailbirds",
  "gaolbreak": "jailbreak",
  "gaolbreaks": "jailbreaks",
  "gaoled": "jailed",
  "gaoler": "jailer",
  "gaolers": "jailers",
  "gaoling": "jailing",
  "gaols": "jails",
  "gases": "gasses",
  "gauge": "gage",
  "gauged": "gaged",
  "gauges": "gages",
  "gauging": "gaging",
  "generalisation": "generalization",
  "generalisations": "generalizations",
  "generalise": "generalize",
  "generalised": "generalized",
  "generalises": "generalizes",
  "generalising": "generalizing",
  "ghettoise": "ghettoize",
  "ghettoised": "ghettoized",
  "ghettoises": "ghettoizes",
  "ghettoising": "ghettoizing",
  "gipsies": "gypsies",
  "glamorise": "glamorize",
  "glamorised": "glamorized",
  "glamorises": "glamorizes",
  "glamorising": "glamorizing",
  "glamour": "glamor",
  "globalisation": "globalization",
  "globalise": "globalize",
  "globalised": "globalized",
  "globalises": "globalizes",
  "globalising": "globalizing",
  "glueing": "gluing",
  "goitre": "goiter",
  "goitres": "goiters",
  "gonorrhoea": "gonorrhea",
  "gormandise": "gormandize",
  "gramme": "gram",
  "grammes": "grams",
  "gravelled": "graveled",
  "grey": "gray",
  "greyed": "grayed",
  "greying": "graying",
  "greyish": "grayish",
  "greyness": "grayness",
  "greys": "grays",
  "grovelled": "groveled",
  "grovelling": "groveling",
  "groyne": "groin",
  "groynes": "groins",
  "gruelling": "grueling",
  "gruellingly": "gruelingly",
  "gryphon": "griffin",
  "gryphons": "griffins",
  "gynaecological": "gynecological",
  "gynaecologist": "gynecologist",
  "gynaecologists": "gynecologists",
  "gynaecology": "gynecology",
  "haematological": "hematological",
  "haematologist": "hematologist",
  "haematologists": "hematologists",
  "haematology": "hematology",
  "haemoglobin": "hemoglobin",
  "haemophilia": "hemophilia",
  "haemophiliac": "hemophiliac",
  "haemophiliacs": "hemophiliacs",
  "haemorrhage": "hemorrhage",
  "haemorrhaged": "hemorrhaged",
  "haemorrhages": "hemorrhages",
  "haemorrhaging": "hemorrhaging",
  "haemorrhoids": "hemorrhoids",
  "harbour": "harbor",
  "harboured": "harbored",
  "harbouring": "harboring",
  "harbours": "harbors",
  "harmonisation": "harmonization",
  "harmonise": "harmonize",
  "harmonised": "harmonized",
  "harmonises": "harmonizes",
  "harmonising": "harmonizing",
  "homoeopath": "homeopath",
  "homoeopathic": "homeopathic",
  "homoeopaths": "homeopaths",
  "homoeopathy": "homeopathy",
  "homogenise": "homogenize",
  "homogenised": "homogenized",
  "homogenises": "homogenizes",
  "homogenising": "homogenizing",
  "honour": "honor",
  "honourable": "honorable",
  "honourably": "honorably",
  "honoured": "honored",
  "honouring": "honoring",
  "honours": "honors",
  "hospitalisation": "hospitalization",
  "hospitalise": "hospitalize",
  "hospitalised": "hospitalized",
  "hospitalises": "hospitalizes",
  "hospitalising": "hospitalizing",
  "humanise": "humanize",
  "humanised": "humanized",
  "humanises": "humanizes",
  "humanising": "humanizing",
  "humour": "humor",
  "humoured": "humored",
  "humouring": "humoring",
  "humourless": "humorless",
  "humours": "humors",
  "hybridise": "hybridize",
  "hybridised": "hybridized",
  "hybridises": "hybridizes",
  "hybridising": "hybridizing",
  "hypnotise": "hypnotize",
  "hypnotised": "hypnotized",
  "hypnotises": "hypnotizes",
  "hypnotising": "hypnotizing",
  "hypothesise": "hypothesize",
  "hypothesised": "hypothesized",
  "hypothesises": "hypothesizes",
  "hypothesising": "hypothesizing",
  "idealisation": "idealization",
  "idealise": "idealize",
  "idealised": "idealized",
  "idealises": "idealizes",
  "idealising": "idealizing",
  "idolise": "idolize",
  "idolised": "idolized",
  "idolises": "idolizes",
  "idolising": "idolizing",
  "immobilisation": "immobilization",
  "immobilise": "immobilize",
  "immobilised": "immobilized",
  "immobiliser": "immobilizer",
  "immobilisers": "immobilizers",
  "immobilises": "immobilizes",
  "immobilising": "immobilizing",
  "immortalise": "immortalize",
  "immortalised": "immortalized",
  "immortalises": "immortalizes",
  "immortalising": "immortalizing",
  "immunisation": "immunization",
  "immunise": "immunize",
  "immunised": "immunized",
  "immunises": "immunizes",
  "immunising": "immunizing",
  "impanelled": "impaneled",
  "impanelling": "impaneling",
  "imperilled": "imperiled",
  "imperilling": "imperiling",
  "individualise": "individualize",
  "individualised": "individualized",
  "individualises": "individualizes",
  "individualising": "individualizing",
  "industrialise": "industrialize",
  "industrialised": "industrialized",
  "industrialises": "industrializes",
  "industrialising": "industrializing",
  "inflexion": "inflection",
  "inflexions": "inflections",
  "initialisation": "initialization",
  "initialisations": "initializations",
  "initialise": "initialize",
  "initialised": "initialized",
  "initialises": "initializes",
  "initialising": "initializing",
  "initialled": "initialed",
  "initialling": "initialing",
  "instal": "install",
  "instalment": "installment",
  "instalments": "installments",
  "instals": "installs",
  "instil": "instill",
  "instils": "instills",
  "institutionalisation": "institutionalization",
  "institutionalise": "institutionalize",
  "institutionalised": "institutionalized",
  "institutionalises": "institutionalizes",
  "institutionalising": "institutionalizing",
  "intellectualise": "intellectualize",
  "intellectualised": "intellectualized",
  "intellectualises": "intellectualizes",
  "intellectualising": "intellectualizing",
  "interiorise": "interiorize",
  "internalisation": "internalization",
  "internalise": "internalize",
  "internalised": "internalized",
  "internalises": "internalizes",
  "internalising": "internalizing",
  "internationalisation": "internationalization",
  "internationalise": "internationalize",
  "internationalised": "internationalized",
  "internationalises": "internationalizes",
  "internationalising": "internationalizing",
  "ionisation": "ionization",
  "ionise": "ionize",
  "ionised": "ionized",
  "ioniser": "ionizer",
  "ionisers": "ionizers",
  "ionises": "ionizes",
  "ionising": "ionizing",
  "italicise": "italicize",
  "italicised": "italicized",
  "italicises": "italicizes",
  "italicising": "italicizing",
  "itemise": "itemize",
  "itemised": "itemized",
  "itemises": "itemizes",
  "itemising": "itemizing",
  "jeopardise": "jeopardize",
  "jeopardised": "jeopardized",
  "jeopardises": "jeopardizes",
  "jeopardising": "jeopardizing",
  "jewelled": "jeweled",
  "jeweller": "jeweler",
  "jewellers": "jewelers",
  "jewellery": "jewelry",
  "judgement": "judgment",
  "kilogramme": "kilogram",
  "kilogrammes": "kilograms",
  "kilometre": "kilometer",
  "kilometres": "kilometers",
  "labelled": "labeled",
  "labelling": "labeling",
  "labour": "labor",
  "laboured": "labored",
  "labourer": "laborer",
  "labourers": "laborers",
  "labouring": "laboring",
  "labours": "labors",
  "lacklustre": "lackluster",
  "laith": "lathe",
  "legalisation": "legalization",
  "legalise": "legalize",
  "legalised": "legalized",
  "legalises": "legalizes",
  "legalising": "legalizing",
  "legitimise": "legitimize",
  "legitimised": "legitimized",
  "legitimises": "legitimizes",
  "legitimising": "legitimizing",
  "leuc": "leuk-",
  "leukaemia": "leukemia",
  "leukaemogenesis": "leukemogenesis",
  "levelled": "leveled",
  "leveller": "leveler",
  "levellers": "levelers",
  "levelling": "leveling",
  "libelled": "libeled",
  "libelling": "libeling",
  "libellous": "libelous",
  "liberalisation": "liberalization",
  "liberalise": "liberalize",
  "liberalised": "liberalized",
  "liberalises": "liberalizes",
  "liberalising": "liberalizing",
  "licence": "license",
  "licenced": "licensed",
  "licences": "licenses",
  "licencing": "licensing",
  "likeable": "likable",
  "lionisation": "lionization",
  "lionise": "lionize",
  "lionised": "lionized",
  "lionises": "lionizes",
  "lionising": "lionizing",
  "liquidise": "liquidize",
  "liquidised": "liquidized",
  "liquidiser": "liquidizer",
  "liquidisers": "liquidizers",
  "liquidises": "liquidizes",
  "liquidising": "liquidizing",
  "liquorice": "licorice",
  "litre": "liter",
  "litres": "liters",
  "localise": "localize",
  "localised": "localized",
  "localises": "localizes",
  "localising": "localizing",
  "louvre": "louver",
  "louvred": "louvered",
  "louvres": "louvers",
  "lustre": "luster",
  "lyophilise": "lyophilize",
  "magnetise": "magnetize",
  "magnetised": "magnetized",
  "magnetises": "magnetizes",
  "magnetising": "magnetizing",
  "manoeuvrability": "maneuverability",
  "manoeuvrable": "maneuverable",
  "manoeuvre": "maneuver",
  "manoeuvred": "maneuvered",
  "manoeuvres": "maneuvers",
  "manoeuvring": "maneuvering",
  "manoeuvrings": "maneuverings",
  "marbleise": "marbleize",
  "marginalisation": "marginalization",
  "marginalise": "marginalize",
  "marginalised": "marginalized",
  "marginalises": "marginalizes",
  "marginalising": "marginalizing",
  "marshalled": "marshaled",
  "marshalling": "marshaling",
  "marvelled": "marveled",
  "marvelling": "marveling",
  "marvellous": "marvelous",
  "marvellously": "marvelously",
  "materialisation": "materialization",
  "materialise": "materialize",
  "materialised": "materialized",
  "materialises": "materializes",
  "materialising": "materializing",
  "maximisation": "maximization",
  "maximise": "maximize",
  "maximised": "maximized",
  "maximises": "maximizes",
  "maximising": "maximizing",
  "meagre": "meager",
  "mechanisation": "mechanization",
  "mechanise": "mechanize",
  "mechanised": "mechanized",
  "mechanises": "mechanizes",
  "mechanising": "mechanizing",
  "mediaeval": "medieval",
  "memorialise": "memorialize",
  "memorialised": "memorialized",
  "memorialises": "memorializes",
  "memorialising": "memorializing",
  "memorise": "memorize",
  "memorised": "memorized",
  "memorises": "memorizes",
  "memorising": "memorizing",
  "mercerise": "mercerize",
  "mesmerise": "mesmerize",
  "mesmerised": "mesmerized",
  "mesmerises": "mesmerizes",
  "mesmerising": "mesmerizing",
  "metabolise": "metabolize",
  "metabolised": "metabolized",
  "metabolises": "metabolizes",
  "metabolising": "metabolizing",
  "methodise": "methodize",
  "metre": "meter",
  "metres": "meters",
  "micrometre": "micrometer",
  "micrometres": "micrometers",
  "militarise": "militarize",
  "militarised": "militarized",
  "militarises": "militarizes",
  "militarising": "militarizing",
  "milligramme": "milligram",
  "milligrammes": "milligrams",
  "millilitre": "milliliter",
  "millilitres": "milliliters",
  "millimetre": "millimeter",
  "millimetres": "millimeters",
  "mineralise": "mineralize",
  "miniaturisation": "miniaturization",
  "miniaturise": "miniaturize",
  "miniaturised": "miniaturized",
  "miniaturises": "miniaturizes",
  "miniaturising": "miniaturizing",
  "minibuses": "minibusses",
  "minimise": "minimize",
  "minimised": "minimized",
  "minimises": "minimizes",
  "minimising": "minimizing",
  "misbehaviour": "misbehavior",
  "misdemeanour": "misdemeanor",
  "misdemeanours": "misdemeanors",
  "misspelt": "misspelled",
  "mitre": "miter",
  "mitres": "miters",
  "mobilisation": "mobilization",
  "mobilise": "mobilize",
  "mobilised": "mobilized",
  "mobilises": "mobilizes",
  "mobilising": "mobilizing",
  "modelled": "modeled",
  "modeller": "modeler",
  "modellers": "modelers",
  "modelling": "modeling",
  "modernisation": "modernization",
  "modernise": "modernize",
  "modernised": "modernized",
  "modernises": "modernizes",
  "modernising": "modernizing",
  "moisturise": "moisturize",
  "moisturised": "moisturized",
  "moisturiser": "moisturizer",
  "moisturisers": "moisturizers",
  "moisturises": "moisturizes",
  "moisturising": "moisturizing",
  "moka": "mocha",
  "monologue": "monolog",
  "monologues": "monologs",
  "monopolisation": "monopolization",
  "monopolise": "monopolize",
  "monopolised": "monopolized",
  "monopolises": "monopolizes",
  "monopolising": "monopolizing",
  "moralise": "moralize",
  "moralised": "moralized",
  "moralises": "moralizes",
  "moralising": "moralizing",
  "motorise": "motorize",
  "motorised": "motorized",
  "mould": "mold",
  "moulded": "molded",
  "moulder": "molder",
  "mouldered": "moldered",
  "mouldering": "moldering",
  "moulders": "molders",
  "mouldier": "moldier",
  "mouldiest": "moldiest",
  "moulding": "molding",
  "mouldings": "moldings",
  "moulds": "molds",
  "mouldy": "moldy",
  "moult": "molt",
  "moulted": "molted",
  "moulting": "molting",
  "moults": "molts",
  "moustache": "mustache",
  "moustached": "mustached",
  "moustaches": "mustaches",
  "moustachioed": "mustachioed",
  "multicoloured": "multicolored",
  "musicalise": "musicalize",
  "nationalisation": "nationalization",
  "nationalisations": "nationalizations",
  "nationalise": "nationalize",
  "nationalised": "nationalized",
  "nationalises": "nationalizes",
  "nationalising": "nationalizing",
  "naturalisation": "naturalization",
  "naturalise": "naturalize",
  "naturalised": "naturalized",
  "naturalises": "naturalizes",
  "naturalising": "naturalizing",
  "neighbour": "neighbor",
  "neighbourhood": "neighborhood",
  "neighbourhoods": "neighborhoods",
  "neighbouring": "neighboring",
  "neighbourliness": "neighborliness",
  "neighbourly": "neighborly",
  "neighbours": "neighbors",
  "neutralisation": "neutralization",
  "neutralise": "neutralize",
  "neutralised": "neutralized",
  "neutralises": "neutralizes",
  "neutralising": "neutralizing",
  "normalisation": "normalization",
  "normalise": "normalize",
  "normalised": "normalized",
  "normalises": "normalizes",
  "normalising": "normalizing",
  "odour": "odor",
  "odourless": "odorless",
  "odours": "odors",
  "oecumenical": "ecumenical",
  "oedema": "edema",
  "oesophagus": "esophagus",
  "oesophaguses": "esophaguses",
  "oestr-": "estr-",
  "oestrogen": "estrogen",
  "offence": "offense",
  "offences": "offenses",
  "omelette": "omelet",
  "omelettes": "omelets",
  "optimisation": "optimization",
  "optimisations": "optimizations",
  "optimise": "optimize",
  "optimised": "optimized",
  "optimises": "optimizes",
  "optimising": "optimizing",
  "organisation": "organization",
  "organisational": "organizational",
  "organisations": "organizations",
  "organise": "organize",
  "organised": "organized",
  "organiser": "organizer",
  "organisers": "organizers",
  "organises": "organizes",
  "organising": "organizing",
  "orthopaedic": "orthopedic",
  "orthopaedics": "orthopedics",
  "ostracise": "ostracize",
  "ostracised": "ostracized",
  "ostracises": "ostracizes",
  "ostracising": "ostracizing",
  "outmanoeuvre": "outmaneuver",
  "outmanoeuvred": "outmaneuvered",
  "outmanoeuvres": "outmaneuvers",
  "outmanoeuvring": "outmaneuvering",
  "overemphasise": "overemphasize",
  "overemphasised": "overemphasized",
  "overemphasises": "overemphasizes",
  "overemphasising": "overemphasizing",
  "oxidisation": "oxidization",
  "oxidise": "oxidize",
  "oxidised": "oxidized",
  "oxidises": "oxidizes",
  "oxidising": "oxidizing",
  "paederast": "pederast",
  "paederasts": "pederasts",
  "paediatric": "pediatric",
  "paediatrician": "pediatrician",
  "paediatricians": "pediatricians",
  "paediatrics": "pediatrics",
  "paedophile": "pedophile",
  "paedophiles": "pedophiles",
  "paedophilia": "pedophilia",
  "paeony": "peony",
  "palaeo-": "paleo-",
  "palaeolithic": "paleolithic",
  "palaeontologist": "paleontologist",
  "palaeontologists": "paleontologists",
  "palaeontology": "paleontology",
  "palletise": "palletize",
  "panelled": "paneled",
  "panelling": "paneling",
  "panellist": "panelist",
  "panellists": "panelists",
  "paraesthesia": "paresthesia",
  "paralyse": "paralyze",
  "paralysed": "paralyzed",
  "paralyses": "paralyzes",
  "paralysing": "paralyzing",
  "parametrisation": "parametrization",
  "parametrisations": "parametrizations",
  "parametrise": "parametrize",
  "parcelled": "parceled",
  "parcelling": "parceling",
  "parlour": "parlor",
  "parlours": "parlors",
  "particularise": "particularize",
  "particularised": "particularized",
  "particularises": "particularizes",
  "particularising": "particularizing",
  "passivisation": "passivization",
  "passivise": "passivize",
  "passivised": "passivized",
  "passivises": "passivizes",
  "passivising": "passivizing",
  "pasteurisation": "pasteurization",
  "pasteurise": "pasteurize",
  "pasteurised": "pasteurized",
  "pasteurises": "pasteurizes",
  "pasteurising": "pasteurizing",
  "patronise": "patronize",
  "patronised": "patronized",
  "patronises": "patronizes",
  "patronising": "patronizing",
  "patronisingly": "patronizingly",
  "pedalled": "pedaled",
  "pedalling": "pedaling",
  "pedestrianisation": "pedestrianization",
  "pedestrianise": "pedestrianize",
  "pedestrianised": "pedestrianized",
  "pedestrianises": "pedestrianizes",
  "pedestrianising": "pedestrianizing",
  "pelletise": "pelletize",
  "penalise": "penalize",
  "penalised": "penalized",
  "penalises": "penalizes",
  "penalising": "penalizing",
  "pencilled": "penciled",
  "pencilling": "penciling",
  "personalise": "personalize",
  "personalised": "personalized",
  "personalises": "personalizes",
  "personalising": "personalizing",
  "pharmacopoeia": "pharmacopeia",
  "pharmacopoeias": "pharmacopeias",
  "philosophise": "philosophize",
  "philosophised": "philosophized",
  "philosophises": "philosophizes",
  "philosophising": "philosophizing",
  "philtre": "filter",
  "philtres": "filters",
  "phoney": "phony",
  "plagiarise": "plagiarize",
  "plagiarised": "plagiarized",
  "plagiarises": "plagiarizes",
  "plagiarising": "plagiarizing",
  "plough": "plow",
  "ploughed": "plowed",
  "ploughing": "plowing",
  "ploughman": "plowman",
  "ploughmen": "plowmen",
  "ploughs": "plows",
  "ploughshare": "plowshare",
  "ploughshares": "plowshares",
  "polarisation": "polarization",
  "polarise": "polarize",
  "polarised": "polarized",
  "polarises": "polarizes",
  "polarising": "polarizing",
  "politicisation": "politicization",
  "politicise": "politicize",
  "politicised": "politicized",
  "politicises": "politicizes",
  "politicising": "politicizing",
  "popularisation": "popularization",
  "popularise": "popularize",
  "popularised": "popularized",
  "popularises": "popularizes",
  "popularising": "popularizing",
  "pouffe": "pouf",
  "pouffes": "poufs",
  "practise": "practice",
  "practised": "practiced",
  "practises": "practices",
  "practising": "practicing",
  "praesidium": "presidium",
  "praesidiums": "presidiums",
  "pressurisation": "pressurization",
  "pressurise": "pressurize",
  "pressurised": "pressurized",
  "pressurises": "pressurizes",
  "pressurising": "pressurizing",
  "pretence": "pretense",
  "pretences": "pretenses",
  "preventor": "preventer",
  "primaeval": "primeval",
  "prioritisation": "prioritization",
  "prioritise": "prioritize",
  "prioritised": "prioritized",
  "prioritises": "prioritizes",
  "prioritising": "prioritizing",
  "privatisation": "privatization",
  "privatisations": "privatizations",
  "privatise": "privatize",
  "privatised": "privatized",
  "privatises": "privatizes",
  "privatising": "privatizing",
  "professionalisation": "professionalization",
  "professionalise": "professionalize",
  "professionalised": "professionalized",
  "professionalises": "professionalizes",
  "professionalising": "professionalizing",
  "programme": "program",
  "programmes": "programs",
  "proletarianise": "proletarianize",
  "prologue": "prolog",
  "prologues": "prologs",
  "propagandise": "propagandize",
  "propagandised": "propagandized",
  "propagandises": "propagandizes",
  "propagandising": "propagandizing",
  "proselytise": "proselytize",
  "proselytised": "proselytized",
  "proselytiser": "proselytizer",
  "proselytisers": "proselytizers",
  "proselytises": "proselytizes",
  "proselytising": "proselytizing",
  "psychoanalyse": "psychoanalyze",
  "psychoanalysed": "psychoanalyzed",
  "psychoanalyses": "psychoanalyzes",
  "psychoanalysing": "psychoanalyzing",
  "publicise": "publicize",
  "publicised": "publicized",
  "publicises": "publicizes",
  "publicising": "publicizing",
  "pulverisation": "pulverization",
  "pulverise": "pulverize",
  "pulverised": "pulverized",
  "pulverises": "pulverizes",
  "pulverising": "pulverizing",
  "pummelled": "pummel",
  "pummelling": "pummeled",
  "pyjama": "pajama",
  "pyjamas": "pajamas",
  "pzazz": "pizzazz",
  "quarrelled": "quarreled",
  "quarrelling": "quarreling",
  "radicalise": "radicalize",
  "radicalised": "radicalized",
  "radicalises": "radicalizes",
  "radicalising": "radicalizing",
  "rancour": "rancor",
  "randomise": "randomize",
  "randomised": "randomized",
  "randomises": "randomizes",
  "randomising": "randomizing",
  "rationalisation": "rationalization",
  "rationalisations": "rationalizations",
  "rationalise": "rationalize",
  "rationalised": "rationalized",
  "rationalises": "rationalizes",
  "rationalising": "rationalizing",
  "ravelled": "raveled",
  "ravelling": "raveling",
  "realisable": "realizable",
  "realisation": "realization",
  "realisations": "realizations",
  "realise": "realize",
  "realised": "realized",
  "realises": "realizes",
  "realising": "realizing",
  "recognisable": "recognizable",
  "recognisably": "recognizably",
  "recognisance": "recognizance",
  "recognise": "recognize",
  "recognised": "recognized",
  "recognises": "recognizes",
  "recognising": "recognizing",
  "reconnoitre": "reconnoiter",
  "reconnoitred": "reconnoitered",
  "reconnoitres": "reconnoiters",
  "reconnoitring": "reconnoitering",
  "reflexion": "reflection",
  "refuelled": "refueled",
  "refuelling": "refueling",
  "regularisation": "regularization",
  "regularise": "regularize",
  "regularised": "regularized",
  "regularises": "regularizes",
  "regularising": "regularizing",
  "remodelled": "remodeled",
  "remodelling": "remodeling",
  "remould": "remold",
  "remoulded": "remolded",
  "remoulding": "remolding",
  "remoulds": "remolds",
  "reorganisation": "reorganization",
  "reorganisations": "reorganizations",
  "reorganise": "reorganize",
  "reorganised": "reorganized",
  "reorganises": "reorganizes",
  "reorganising": "reorganizing",
  "revelled": "reveled",
  "reveller": "reveler",
  "revellers": "revelers",
  "revelling": "reveling",
  "revitalise": "revitalize",
  "revitalised": "revitalized",
  "revitalises": "revitalizes",
  "revitalising": "revitalizing",
  "revolutionise": "revolutionize",
  "revolutionised": "revolutionized",
  "revolutionises": "revolutionizes",
  "revolutionising": "revolutionizing",
  "rhapsodise": "rhapsodize",
  "rhapsodised": "rhapsodized",
  "rhapsodises": "rhapsodizes",
  "rhapsodising": "rhapsodizing",
  "rigour": "rigor",
  "rigours": "rigors",
  "ritualised": "ritualized",
  "rivalled": "rivaled",
  "rivalling": "rivaling",
  "romanticise": "romanticize",
  "romanticised": "romanticized",
  "romanticises": "romanticizes",
  "romanticising": "romanticizing",
  "rumour": "rumor",
  "rumoured": "rumored",
  "rumours": "rumors",
  "sabre": "saber",
  "sabres": "sabers",
  "saleroom": "salesroom",
  "saltpetre": "saltpeter",
  "sanitise": "sanitize",
  "sanitised": "sanitized",
  "sanitises": "sanitizes",
  "sanitising": "sanitizing",
  "satirise": "satirize",
  "satirised": "satirized",
  "satirises": "satirizes",
  "satirising": "satirizing",
  "saviour": "savior",
  "saviours": "saviors",
  "savour": "savor",
  "savoured": "savored",
  "savouries": "savories",
  "savouring": "savoring",
  "savours": "savors",
  "savoury": "savory",
  "scandalise": "scandalize",
  "scandalised": "scandalized",
  "scandalises": "scandalizes",
  "scandalising": "scandalizing",
  "sceptic": "skeptic",
  "sceptical": "skeptical",
  "sceptically": "skeptically",
  "scepticism": "skepticism",
  "sceptics": "skeptics",
  "sceptre": "scepter",
  "sceptres": "scepters",
  "scrutinise": "scrutinize",
  "scrutinised": "scrutinized",
  "scrutinises": "scrutinizes",
  "scrutinising": "scrutinizing",
  "secularisation": "secularization",
  "secularise": "secularize",
  "secularised": "secularized",
  "secularises": "secularizes",
  "secularising": "secularizing",
  "sensationalise": "sensationalize",
  "sensationalised": "sensationalized",
  "sensationalises": "sensationalizes",
  "sensationalising": "sensationalizing",
  "sensitisation": "sensitization",
  "sensitise": "sensitize",
  "sensitised": "sensitized",
  "sensitises": "sensitizes",
  "sensitising": "sensitizing",
  "sentimentalise": "sentimentalize",
  "sentimentalised": "sentimentalized",
  "sentimentalises": "sentimentalizes",
  "sentimentalising": "sentimentalizing",
  "sepulchre": "sepulcher",
  "sepulchres": "sepulchers",
  "serialisation": "serialization",
  "serialisations": "serializations",
  "serialise": "serialize",
  "serialised": "serialized",
  "serialises": "serializes",
  "serialising": "serializing",
  "sermonise": "sermonize",
  "sermonised": "sermonized",
  "sermonises": "sermonizes",
  "sermonising": "sermonizing",
  "sheikh": "sheik",
  "shovelled": "shoveled",
  "shovelling": "shoveling",
  "shrivelled": "shriveled",
  "shrivelling": "shriveling",
  "signalise": "signalize",
  "signalised": "signalized",
  "signalises": "signalizes",
  "signalising": "signalizing",
  "signalled": "signaled",
  "signalling": "signaling",
  "sleugh": "slough",
  "smoulder": "smolder",
  "smouldered": "smoldered",
  "smouldering": "smoldering",
  "smoulders": "smolders",
  "snivelled": "sniveled",
  "snivelling": "sniveling",
  "snorkelled": "snorkeled",
  "snorkelling": "snorkeling",
  "snowplough": "snowplow",
  "snowploughs": "snowplow",
  "socialisation": "socialization",
  "socialise": "socialize",
  "socialised": "socialized",
  "socialises": "socializes",
  "socialising": "socializing",
  "sodomise": "sodomize",
  "sodomised": "sodomized",
  "sodomises": "sodomizes",
  "sodomising": "sodomizing",
  "solemnise": "solemnize",
  "solemnised": "solemnized",
  "solemnises": "solemnizes",
  "solemnising": "solemnizing",
  "soliloquise": "soliloquize",
  "solubilise": "solubilize",
  "sombre": "somber",
  "specialisation": "specialization",
  "specialisations": "specializations",
  "specialise": "specialize",
  "specialised": "specialized",
  "specialises": "specializes",
  "specialising": "specializing",
  "spectre": "specter",
  "spectres": "specters",
  "spiralled": "spiraled",
  "spiralling": "spiraling",
  "splendour": "splendor",
  "splendours": "splendors",
  "squirrelled": "squirreled",
  "squirrelling": "squirreling",
  "stabilisation": "stabilization",
  "stabilise": "stabilize",
  "stabilised": "stabilized",
  "stabiliser": "stabilizer",
  "stabilisers": "stabilizers",
  "stabilises": "stabilizes",
  "stabilising": "stabilizing",
  "standardisation": "standardization",
  "standardise": "standardize",
  "standardised": "standardized",
  "standardises": "standardizes",
  "standardising": "standardizing",
  "steatorrhoea": "steatorrhea",
  "stencilled": "stenciled",
  "stencilling": "stenciling",
  "sterilisation": "sterilization",
  "sterilisations": "sterilizations",
  "sterilise": "sterilize",
  "sterilised": "sterilized",
  "steriliser": "sterilizer",
  "sterilisers": "sterilizers",
  "sterilises": "sterilizes",
  "sterilising": "sterilizing",
  "stigmatisation": "stigmatization",
  "stigmatise": "stigmatize",
  "stigmatised": "stigmatized",
  "stigmatises": "stigmatizes",
  "stigmatising": "stigmatizing",
  "stoicheiometry": "stoichiometry",
  "stylise": "stylize",
  "subjectivise": "subjectivize",
  "subsidisation": "subsidization",
  "subsidise": "subsidize",
  "subsidised": "subsidized",
  "subsidiser": "subsidizer",
  "subsidisers": "subsidizers",
  "subsidises": "subsidizes",
  "subsidising": "subsidizing",
  "suburbanise": "suburbanize",
  "succour": "succor",
  "succoured": "succored",
  "succouring": "succoring",
  "succours": "succors",
  "sulphate": "sulfate",
  "sulphates": "sulfates",
  "sulphide": "sulfide",
  "sulphides": "sulfides",
  "sulphur": "sulfur",
  "sulphurous": "sulfurous",
  "summarise": "summarize",
  "summarised": "summarized",
  "summarises": "summarizes",
  "summarising": "summarizing",
  "swivelled": "swiveled",
  "swivelling": "swiveling",
  "symbolise": "symbolize",
  "symbolised": "symbolized",
  "symbolises": "symbolizes",
  "symbolising": "symbolizing",
  "sympathise": "sympathize",
  "sympathised": "sympathized",
  "sympathiser": "sympathizer",
  "sympathisers": "sympathizers",
  "sympathises": "sympathizes",
  "sympathising": "sympathizing",
  "synaesthesia": "synesthesia",
  "synchronisation": "synchronization",
  "synchronise": "synchronize",
  "synchronised": "synchronized",
  "synchronises": "synchronizes",
  "synchronising": "synchronizing",
  "syntagm": "syntagma",
  "synthesise": "synthesize",
  "synthesised": "synthesized",
  "synthesiser": "synthesizer",
  "synthesisers": "synthesizers",
  "synthesises": "synthesizes",
  "synthesising": "synthesizing",
  "syphon": "siphon",
  "syphoned": "siphoned",
  "syphoning": "siphoning",
  "syphons": "siphons",
  "systematisation": "systematization",
  "systematise": "systematize",
  "systematised": "systematized",
  "systematises": "systematizes",
  "systematising": "systematizing",
  "tantalise": "tantalize",
  "tantalised": "tantalized",
  "tantalises": "tantalizes",
  "tantalising": "tantalizing",
  "tantalisingly": "tantalizingly",
  "tasselled": "tasseled",
  "technicolour": "technicolor",
  "temporise": "temporize",
  "temporised": "temporized",
  "temporises": "temporizes",
  "temporising": "temporizing",
  "tendencious": "tendentious",
  "tenderise": "tenderize",
  "tenderised": "tenderized",
  "tenderises": "tenderizes",
  "tenderising": "tenderizing",
  "terrorise": "terrorize",
  "terrorised": "terrorized",
  "terrorises": "terrorizes",
  "terrorising": "terrorizing",
  "thalassaemia": "thalassemia",
  "theatre": "theater",
  "theatregoer": "theatergoer",
  "theatregoers": "theatergoers",
  "theatres": "theaters",
  "theorise": "theorize",
  "theorised": "theorized",
  "theorises": "theorizes",
  "theorising": "theorizing",
  "tonne": "ton",
  "tonnes": "tons",
  "towelled": "toweled",
  "towelling": "toweling",
  "toxaemia": "toxemia",
  "tranquillise": "tranquilize",
  "tranquillised": "tranquilized",
  "tranquilliser": "tranquilizer",
  "tranquillisers": "tranquilizers",
  "tranquillises": "tranquilizes",
  "tranquillising": "tranquilizing",
  "tranquillity": "tranquility",
  "tranquillize": "tranquilize",
  "tranquillized": "tranquilized",
  "tranquillizer": "tranquilizer",
  "tranquillizers": "tranquilizers",
  "tranquillizes": "tranquilizes",
  "tranquillizing": "tranquilizing",
  "tranquilly": "tranquility",
  "transistorised": "transistorized",
  "traumatise": "traumatize",
  "traumatised": "traumatized",
  "traumatises": "traumatizes",
  "traumatising": "traumatizing",
  "travelled": "traveled",
  "traveller": "traveler",
  "travellers": "travelers",
  "travelling": "traveling",
  "travelogue": "travelog",
  "travelogues": "travelogs",
  "trialled": "trialed",
  "trialling": "trialing",
  "tricolour": "tricolor",
  "tricolours": "tricolors",
  "trivialise": "trivialize",
  "trivialised": "trivialized",
  "trivialises": "trivializes",
  "trivialising": "trivializing",
  "tumour": "tumor",
  "tumours": "tumors",
  "tunnelled": "tunneled",
  "tunnelling": "tunneling",
  "tyrannise": "tyrannize",
  "tyrannised": "tyrannized",
  "tyrannises": "tyrannizes",
  "tyrannising": "tyrannizing",
  "tyre": "tire",
  "tyres": "tires",
  "unauthorised": "unauthorized",
  "uncivilised": "uncivilized",
  "underutilised": "underutilized",
  "unequalled": "unequaled",
  "unfavourable": "unfavorable",
  "unfavourably": "unfavorably",
  "unionisation": "unionization",
  "unionise": "unionize",
  "unionised": "unionized",
  "unionises": "unionizes",
  "unionising": "unionizing",
  "unorganised": "unorganized",
  "unravelled": "unraveled",
  "unravelling": "unraveling",
  "unrecognisable": "unrecognizable",
  "unrecognised": "unrecognized",
  "unrivalled": "unrivaled",
  "unsavoury": "unsavory",
  "untrammelled": "untrammeled",
  "urbanisation": "urbanization",
  "urbanise": "urbanize",
  "urbanised": "urbanized",
  "urbanises": "urbanizes",
  "urbanising": "urbanizing",
  "utilisable": "utilizable",
  "utilisation": "utilization",
  "utilise": "utilize",
  "utilised": "utilized",
  "utilises": "utilizes",
  "utilising": "utilizing",
  "valour": "valor",
  "vandalise": "vandalize",
  "vandalised": "vandalized",
  "vandalises": "vandalizes",
  "vandalising": "vandalizing",
  "vaporisation": "vaporization",
  "vaporise": "vaporize",
  "vaporised": "vaporized",
  "vaporises": "vaporizes",
  "vaporising": "vaporizing",
  "vapour": "vapor",
  "vapours": "vapors",
  "verbalise": "verbalize",
  "verbalised": "verbalized",
  "verbalises": "verbalizes",
  "verbalising": "verbalizing",
  "victimisation": "victimization",
  "victimise": "victimize",
  "victimised": "victimized",
  "victimises": "victimizes",
  "victimising": "victimizing",
  "videodisc": "videodisk",
  "videodiscs": "videodisks",
  "vigour": "vigor",
  "visualisation": "visualization",
  "visualisations": "visualizations",
  "visualise": "visualize",
  "visualised": "visualized",
  "visualises": "visualizes",
  "visualising": "visualizing",
  "vocalisation": "vocalization",
  "vocalisations": "vocalizations",
  "vocalise": "vocalize",
  "vocalised": "vocalized",
  "vocalises": "vocalizes",
  "vocalising": "vocalizing",
  "vulcanisate": "vulcanizate",
  "vulcanised": "vulcanized",
  "vulgarisation": "vulgarization",
  "vulgarise": "vulgarize",
  "vulgarised": "vulgarized",
  "vulgarises": "vulgarizes",
  "vulgarising": "vulgarizing",
  "waggon": "wagon",
  "waggons": "wagons",
  "watercolour": "watercolor",
  "watercolours": "watercolors",
  "weaselled": "weaseled",
  "weaselling": "weaseling",
  "westernisation": "westernization",
  "westernise": "westernize",
  "westernised": "westernized",
  "westernises": "westernizes",
  "westernising": "westernizing",
  "womanise": "womanize",
  "womanised": "womanized",
  "womaniser": "womanizer",
  "womanisers": "womanizers",
  "womanises": "womanizes",
  "womanising": "womanizing",
  "woollen": "woolen",
  "woollens": "woolens",
  "woollies": "woolies",
  "woolly": "wooly",
  "worshipped": "worshiped",
  "worshipper": "worshiper",
  "worshipping": "worshiping",
  "yodelled": "yodeled",
  "yodelling": "yodeling",
  "yoghourt": "yogurt",
  "yoghourts": "yogurts",
  "yoghurt": "yogurt",
  "yoghurts": "yogurts"
}

In [42]:
def replace_all(text, mydict):
    for gb, us in mydict.items():
        text = text.replace(us, gb)
    return text

In [43]:
gb_text = hp_lp_concat['content_cleaned']

In [45]:
hp_lp_concat['content_cleaned_debritish']  = replace_all(hp_lp_concat['content_cleaned'], us2gb)

In [46]:
hp_lp_concat

,title,content,wordCount,country,content_cleaned,peace/non-peace,content_cleaned_debritish
0,MONTHLY SECTOR REPORT: S&P/ASX EMERGING COMPAN...,AUSTRALIAN MONTHLY INDEX REPORT\n\nThe S&P/ASX...,2405,Australia,MONTHLY SECTOR REPORT : / ASX EMERGING COMPANI...,1,MONTHLY SECTOR REPORT : / ASX EMERGING COMPANI...
1,PAPRIKA CLUB RESTAURANT,THE Paprika Club Restaurant offers freshly pre...,120,Australia,"RESTAURANT \n offers freshly prepared, delicio...",1,"RESTAURANT \n offers freshly prepared, delicio..."
2,Bone implants to be made from 3D printed salt,ABSTRACT\n\n\n\nResearchers have developed a n...,545,Australia,Bone implants to be made from 3D printed salt ...,1,Bone implants to be made from 3D printed salt ...
3,"Yes, you're entitled to a free portrait of the...",It's a truth universally acknowledged that a h...,598,Australia,"Yes, you're entitled to free portrait of the. ...",1,"Yes, you're entitled to free portrait of the. ..."
4,Why is Hemsworth wearing nail polish?,If you thought nail polish was only for women ...,729,Australia,Why is wearing nail polish? \n If you thought ...,1,Why is wearing nail polish? \n If you thought ...
...,...,...,...,...,...,...,...
1979718,"Intel Awarded Patent for Systems, Methods and ...",FULL TEXT\n\n\n\nPublication Name: Software Pa...,241,India,\n FULL TEXT \n\n\n\n Publication Name : Softw...,0,\n FULL TEXT \n\n\n\n Publication Name : Softw...
1979719,Morning Alert: Williamson Magor & Co keeps ris...,INDIAN INTRA-DAY STOCK REPORT\n\nWilliamson Ma...,680,India,Morning Alert : keeps rising : up 22.0% in 6 d...,0,Morning Alert : keeps rising : up 22.0% in 6 d...
1979720,"LJUBLJANSKE MLEKARNE, mlekarska industrija, d....","Slovenia, Nov. 14 -- Slovenia based LJUBLJANSK...",68,India,"LJUBLJANSKE MLEKARNE, industrija, d.o.o. contr...",0,"LJUBLJANSKE MLEKARNE, industrija, d.o.o. contr..."
1979721,STEEL AUTHORITY OF INDIA LIMITED (SAIL) Provid...,"India ,April 10 -- Tender No. CCNW/PR11300076...",51,India,STEEL AUTHORITY OF INDIA LIMITED (SAIL) Provid...,0,STEEL AUTHORITY OF INDIA LIMITED (SAIL) Provid...


In [53]:
hp_lp_concat.to_csv('s3://compressed-data-sample/hp_lp_concat.csv')

In [21]:
pandarallel.initialize()

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [25]:
hp_lp_concat['content_cleaned_debritish'] = hp_lp_concat['content_cleaned'].parallel_apply(americanize)

KeyError: 'americanisation'

## Removing duplicates"

In [11]:
def get_dupes(df, col, preview = False, headsize = 10):
    """
    For a given column of a dataframe, returns a series whose index are the values of col and whose values says whether or not that value has a duplicate.
    Optionally prints the head of the series using boolean preview.
    
    args: df (dataframe), col (column name), preview = False (print head), headsize = 10 (length of head to print))
    returns: boolean series indexed by values of df[col]
    """
    duplicates = df[col].value_counts().apply(lambda t: t>1)
    if preview:
        print(df[df[col].apply(lambda ii: duplicates[ii])].sort_values(col).head(headsize))
    return duplicates

In [12]:
#textdupes_lp = get_dupes(lp_df,'content', preview = False)
#print(f'LP duplicates: {textdupes_lp.sum()}')
#print(f'% of LP articles which are duplicates: {textdupes_lp.sum()/lp_df.shape[0]*100:.2f}%')
#del textdupes_lp

textdupes_hp = get_dupes(hp_df,'content', preview = False)
print(f'HP duplicates: {textdupes_hp.sum()}')
print(f'% of HP articles which are duplicates: {textdupes_hp.sum()/hp_df.shape[0]*100:.2f}%')
del textdupes_hp

HP duplicates: 13344
% of HP articles which are duplicates: 1.33%


In [13]:
def duplicated_across_feature(df, col1, col2):
    """Checks if there are duplicates in col1 across different features in col2
     For example, col1 could be the text column, col2 the country column, to check if there are text duplicates
     across different countries.
     
     args: 
         df: dataframe
         col1: column to check for duplicates
         col2: feature to count duplicates against
    returns: 
        dataframe giving count of distinct col2 values where duplicate col1 values occur"""
    col1dupe_col2 = df.groupby(col1)[col2].apply(lambda t: t.unique())
    col1dupe_col2.name = col2+'_list'
    col1dupe_col2 = col1dupe_col2.reset_index()
    col1dupe_col2[col2+'_count'] = col1dupe_col2[col2+'_list'].apply(lambda t:len(t))
    return col1dupe_col2

In [14]:
#lp_countrydupes = duplicated_across_feature(lp_df, 'content', 'country')
#print(f'LP articles duplicated across multiple countries: {(lp_countrydupes.country_count > 1).sum()}')

hp_countrydupes = duplicated_across_feature(hp_df, 'content', 'country')
print(f'HP articles duplicated across multiple countries: {(hp_countrydupes.country_count > 1).sum()}')

HP articles duplicated across multiple countries: 68


In [7]:
########### SKIP !!!!!
#LP

#Values of lp_csv[content] which are duplicated across different countries
lp_dup_drops = lp_countrydupes[lp_countrydupes.country_count > 1].content.values

#Remove lp articles which are duplicates across countries entirely
lp_clean = lp_df[~lp_df.content.isin(lp_dup_drops)].copy()
n = lp_clean.shape[0]

print(f'Removing ALL LP articles duplicated across countries dropped {lp_df.shape[0] - n} rows.')

#then drop remaining (within-country) duplicates (keeping one)
lp_clean.drop_duplicates(inplace = True)
print(f'Dropping LP duplicates (within country, keeping one) removed {n - lp_clean.shape[0]} additional rows.')
n = lp_clean.shape[0]

print(f'LP articles after removing any article duplicated across countries, and all but one remaining (within-country) duplicate: {n}')

del lp_dup_drops, lp_df

NameError: name 'lp_countrydupes' is not defined

In [15]:
#HP

#Values of hp_csv[content] which are duplicated across different countries
hp_dup_drops = hp_countrydupes[hp_countrydupes.country_count > 1].content.values

#Remove hp articles which are duplicates across countries entirely
hp_clean = hp_df[~hp_df.content.isin(hp_dup_drops)].copy()
n = hp_clean.shape[0]

print(f'Removing ALL HP articles duplicated across countries dropped {hp_df.shape[0] - n} rows.')

#then drop remaining (within-country) duplicates (keeping one)
hp_clean.drop_duplicates(inplace = True)
print(f'Dropping HP duplicates (within country, keeping one) removed {n - hp_clean.shape[0]} additional rows.')
n = hp_clean.shape[0]

print(f'HP articles after removing any article duplicated across countries, and all but one remaining (within-country) duplicate: {n}')

del hp_dup_drops, hp_df

Removing ALL HP articles duplicated across countries dropped 144 rows.
Dropping HP duplicates (within country, keeping one) removed 16267 additional rows.
HP articles after removing any article duplicated across countries, and all but one remaining (within-country) duplicate: 983589


In [16]:
hp_clean.reset_index(drop=True,inplace = True)
#4lp_clean.reset_index(drop=True,inplace = True)

## Convert british to american; NER; stop words

In [22]:
CONVERT_DICT_URL = "https://raw.githubusercontent.com/hyperreality/American-British-English-Translator/master/data/british_spellings.json"
british_to_american_dict = requests.get(CONVERT_DICT_URL).json()
REMOVED_ENTITY_TYPE = set(['PERSON', 'GPE', 'ORG'])
SENT_TO_REMOVE = re.compile('For any query with respect to this article or any other content requirement, please contact Editor at')
special_pattern = re.compile(r'(http\S+)|(\S+.com)|(\S*@\S*\s?)|\(\S+ ?: ?\S+\)', flags=re.MULTILINE)
spacy_nlp = spacy.load('en_core_web_sm',  
                       exclude=['senter', 'entity_ruler','textcat', 'sentencizer', 'merge_noun_chunks', 'tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer'])

In [23]:
british_pattern = '|'.join([r'\b'+word+r'\b' for word in british_to_american_dict.keys()])


def americanize(string):
    url ="https://raw.githubusercontent.com/hyperreality/American-British-English-Translator/master/data/british_spellings.json"
    british_to_american_dict = requests.get(url).json()  
    
    for british_word in set([i.lower() for i in re.findall(british_pattern, string, flags=re.IGNORECASE)]):
        string = re.sub(r'\b'+british_word+r'\b', british_to_american_dict[british_word], string, flags=re.IGNORECASE)
  
    return string

In [24]:
americanize(hp_lp_concat.loc[0,'content_cleaned'])

'MONTHLY SECTOR REPORT : / ASX EMERGING COMPANIES INDEX STRENGTHENS ABOVE MOVING AVERAGE (), WITH 50 - DAY MA BEATING ITS 200 - DAY MA, RISES 3.5% January 31, 2020 16:00:00 \n AUSTRALIAN MONTHLY INDEX REPORT \n\n The S&P / ASX Emerging Companies Index rose 54.3 points or 3.5% to 1,593.0 in the month to 31 January, 2020, ending three - month streak of losses. \n Compared with the ALL ORDINARIES INDEX which rose 4.7% (or 318.8 points) for the month, this represented relative price change of -1.2% for the month. \n The index has strengthened above its trend. \n The index value to 200 - day moving average ratio is 1.06, bullish indicator. \n In the past 200 days this ratio has exceeded 1.06 one hundred thirty - one times suggesting further upside. \n The index is trading above both its 200 - day and the 50 - day MA. Further, the 50 - day MA of 1,564.5 is higher than the 200 - day MA of 1,504.3, bullish indicator. \n The 200 - day has increased to 1,504.3, another bullish indicator. \n\n PR

In [52]:
 url ="https://raw.githubusercontent.com/hyperreality/American-British-English-Translator/master/data/british_spellings.json"
british_to_american = requests.get(url).json()    

def americanize2(string):
    for british_spelling, american_spelling in british_to_american.items():
        #string = string.replace(british_spelling, american_spelling) 
        string = re.sub(r'\b'+british_spelling+'\b', american_spelling, string, flags=re.IGNORECASE)
    return string

In [56]:
 url ="https://raw.githubusercontent.com/hyperreality/American-British-English-Translator/master/data/british_spellings.json"
british_to_american = requests.get(url).json()    

def americanize2(string):
    for british_spelling, american_spelling in british_to_american.items():
        #string = string.replace(british_spelling, american_spelling) 
        string = re.sub(r'\b'+british_word+r'\b', british_to_american[british_word], string, flags=re.IGNORECASE)
    return string

In [57]:
americanize2(hp_lp_concat.iloc[0,5])

NameError: name 'british_word' is not defined

In [28]:
import requests

def de_brit(string):
    url ="https://raw.githubusercontent.com/hyperreality/American-British-English-Translator/master/data/british_spellings.json"
    british_to_american_dict = requests.get(url).json()    

    for british_spelling, american_spelling in british_to_american_dict.items():
        string = string.replace(british_spelling, american_spelling)
  
    return string

In [29]:
de_brit(hp_lp_concat.loc[0:'content_cleaned'])

KeyboardInterrupt: 

In [63]:
de_brit(hp_lp_concat.iloc[0,5])

AttributeError: 'numpy.int64' object has no attribute 'replace'

In [13]:
def preprocess(sentence):

    # URL and Email Removal
    # Remove the repeating phrase
    sentence = SENT_TO_REMOVE.sub(' ', sentence)
    
    # Remove website url and email address and Stock Symbols
    sentence = special_pattern.sub('', sentence)
    
    # SKIP
   # sentence = americanize(sentence)
      
    #Identify and Remove NER
    sentence = ' '.join(['%s'%ent for ent in spacy_nlp(sentence) if ent.ent_type_ not in REMOVED_ENTITY_TYPE])
    
    # Remove new line character
    sentences = re.sub('\n', ' ', sentence)
    # Remove single letter word since it doesn't make sense; might be some technical terms that are trimmed off
    sentence = re.sub('\ [a-zA-z]\ ', ' ', sentence, flags=re.IGNORECASE)
     # Remove redundant spaces and unnecessary spaces between punctuations and letters
    sentence = re.sub(' +', ' ', sentence)
    sentence  = re.sub(r'(\s([,.?!%"\']))|(?<=\[|\()(.*?)(?=\)|\])', lambda x: x.group().strip(), sentence)
    
    return sentence

In [21]:
#pandarallel.initialize()

#time_start = time.time()
#lp_clean['content_cleaned'] = lp_clean.parallel_apply(lambda x: preprocess(x['title'] + '\n' + x['content']), axis=1)
#time_end = time.time()

#print(f'Preprocessing Time: {time_end - time_start} seconds')

In [22]:
#lp_clean.to_csv('s3://compressed-data-sample/processed_lp.csv')

In [23]:
pandarallel.initialize()

time_start = time.time()
hp_clean['content_cleaned'] = hp_clean.parallel_apply(lambda x: preprocess(x['title'] + '\n' + x['content']), axis=1)
time_end = time.time()

print(f'Preprocessing Time: {time_end - time_start} seconds')

INFO: Pandarallel will run on 18 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Preprocessing Time: 2599.970057964325 seconds


In [24]:
hp_clean.to_csv('s3://compressed-data-sample/processed_hp.csv')

In [25]:
hp_clean.head()

,title,content,wordCount,country,content_cleaned
0,MONTHLY SECTOR REPORT: S&P/ASX EMERGING COMPAN...,AUSTRALIAN MONTHLY INDEX REPORT\n\nThe S&P/ASX...,2405,Australia,MONTHLY SECTOR REPORT : / ASX EMERGING COMPANI...
1,PAPRIKA CLUB RESTAURANT,THE Paprika Club Restaurant offers freshly pre...,120,Australia,"RESTAURANT \n offers freshly prepared, delicio..."
2,Bone implants to be made from 3D printed salt,ABSTRACT\n\n\n\nResearchers have developed a n...,545,Australia,Bone implants to be made from 3D printed salt ...
3,"Yes, you're entitled to a free portrait of the...",It's a truth universally acknowledged that a h...,598,Australia,"Yes, you're entitled to free portrait of the. ..."
4,Why is Hemsworth wearing nail polish?,If you thought nail polish was only for women ...,729,Australia,Why is wearing nail polish? \n If you thought ...
